In [1]:
import pprint

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
import tensorflow_ranking as tfr
import tensorflow_recommenders as tfrs

In [3]:
import pprint

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
import tensorflow_ranking as tfr
import tensorflow_recommenders as tfrs

In [5]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})
movies = movies.map(lambda x: x["movie_title"])

unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2022-05-17 16:21:02.479750: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


### Data preprocessing

In [6]:
tf.random.set_seed(42)

# Split between train and tests sets, as before.
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [15]:
def printds(ds, n=5):
    for example in ds.take(n):
        pprint.pprint(example)

In [16]:
printds(train)

{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Postman, The (1997)'>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'681'>,
 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=4.0>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Clueless (1995)'>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'442'>,
 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=2.0>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Maltese Falcon, The (1941)'>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'932'>,
 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=5.0>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'His Girl Friday (1940)'>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'506'>,
 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=5.0>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Quiz Show (1994)'>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'18'>,
 'user_r

In [18]:
import array
import collections

from typing import Dict, List, Optional, Text, Tuple

In [19]:
def _create_feature_dict() -> Dict[Text, List[tf.Tensor]]:
  """Helper function for creating an empty feature dict for defaultdict."""
  return {"movie_title": [], "user_rating": []}

In [20]:
example_lists_by_user = collections.defaultdict(_create_feature_dict)

In [21]:
example_lists_by_user

defaultdict(<function __main__._create_feature_dict() -> Dict[str, List[tensorflow.python.framework.ops.Tensor]]>,
            {})

In [22]:
movie_title_vocab = set()

In [23]:
for example in train:
    user_id = example["user_id"].numpy()
    example_lists_by_user[user_id]["movie_title"].append(example["movie_title"])
    example_lists_by_user[user_id]["user_rating"].append(example["user_rating"])
    movie_title_vocab.add(example["movie_title"].numpy())

In [24]:
tensor_slices = {"user_id": [], "movie_title": [], "user_rating": []}

In [30]:
num_list_per_user = 50
num_examples_per_list = 5

In [33]:
def _sample_list(
        feature_lists: Dict[Text, List[tf.Tensor]],
        num_examples_per_list: int,
        random_state: Optional[np.random.RandomState] = None,
) -> Tuple[tf.Tensor, tf.Tensor]:
    """Function for sampling a list example from given feature lists."""
    if random_state is None:
        random_state = np.random.RandomState()

    sampled_indices = random_state.choice(
        range(len(feature_lists["movie_title"])),
        size = num_examples_per_list,
        replace = False,
    )
    sampled_movie_titles = [
        feature_lists["movie_title"][idx] for idx in sampled_indices
    ]
    sampled_ratings = [
        feature_lists["user_rating"][idx] for idx in sampled_indices
    ]

    return (
        tf.concat(sampled_movie_titles, 0),
        tf.concat(sampled_ratings, 0),
    )

In [35]:
seed = 42

In [36]:
for user_id, feature_lists in example_lists_by_user.items():
    random_state = np.random.RandomState(seed)
    for _ in range(num_list_per_user):

        # Drop the user if they don't have enough ratings.
        if len(feature_lists["movie_title"]) < num_examples_per_list:
            continue

        sampled_movie_titles, sampled_ratings = _sample_list(
            feature_lists,
            num_examples_per_list,
            random_state = random_state,
        )
        tensor_slices["user_id"].append(user_id)
        tensor_slices["movie_title"].append(sampled_movie_titles)
        tensor_slices["user_rating"].append(sampled_ratings)

In [37]:
new_train = tf.data.Dataset.from_tensor_slices(tensor_slices)

In [38]:
printds(new_train)

{'movie_title': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'Postman, The (1997)', b'Liar Liar (1997)', b'Contact (1997)',
       b'Welcome To Sarajevo (1997)',
       b'I Know What You Did Last Summer (1997)'], dtype=object)>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'681'>,
 'user_rating': <tf.Tensor: shape=(5,), dtype=float32, numpy=array([4., 5., 1., 4., 1.], dtype=float32)>}
{'movie_title': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'George of the Jungle (1997)', b'Home Alone 3 (1997)',
       b'Contact (1997)', b'Anastasia (1997)',
       b'Swept from the Sea (1997)'], dtype=object)>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'681'>,
 'user_rating': <tf.Tensor: shape=(5,), dtype=float32, numpy=array([2., 1., 1., 3., 5.], dtype=float32)>}
{'movie_title': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'Conspiracy Theory (1997)', b'Evita (1996)',
       b'Postman, The (1997)', b'Mouse Hunt (1997)', b'Anastasia (1997)'],
      dtype

In [39]:
# We sample 50 lists for each user for the training data. For each list we
# sample 5 movies from the movies the user rated.
# train = tfrs.examples.movielens.sample_listwise(
#     train,
#     num_list_per_user=50,
#     num_examples_per_list=5,
#     seed=42
# )
test = tfrs.examples.movielens.sample_listwise(
    test,
    num_list_per_user=1,
    num_examples_per_list=5,
    seed=42
)

In [40]:
train = new_train

In [41]:
for example in train.take(1):
    pprint.pprint(example)

{'movie_title': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'Postman, The (1997)', b'Liar Liar (1997)', b'Contact (1997)',
       b'Welcome To Sarajevo (1997)',
       b'I Know What You Did Last Summer (1997)'], dtype=object)>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'681'>,
 'user_rating': <tf.Tensor: shape=(5,), dtype=float32, numpy=array([4., 5., 1., 4., 1.], dtype=float32)>}


### Model definition

In [43]:
class RankingModel(tfrs.Model):

	def __init__(self, loss):
		super().__init__()
		embedding_dimension = 32

		# Compute embeddings for users.
		self.user_embeddings = tf.keras.Sequential(
			[
				tf.keras.layers.experimental.preprocessing.StringLookup(
					vocabulary = unique_user_ids
				),
				tf.keras.layers.Embedding(len(unique_user_ids) + 2, embedding_dimension)
			]
		)

		# Compute embeddings for movies.
		self.movie_embeddings = tf.keras.Sequential(
			[
				tf.keras.layers.experimental.preprocessing.StringLookup(
					vocabulary = unique_movie_titles
				),
				tf.keras.layers.Embedding(len(unique_movie_titles) + 2, embedding_dimension)
			]
		)

		# Compute predictions.
		self.score_model = tf.keras.Sequential(
			[
				# Learn multiple dense layers.
				tf.keras.layers.Dense(256, activation = "relu"),
				tf.keras.layers.Dense(64, activation = "relu"),
				# Make rating predictions in the final layer.
				tf.keras.layers.Dense(1)
			]
		)

		self.task = tfrs.tasks.Ranking(
			loss = loss,
			metrics = [
				tfr.keras.metrics.NDCGMetric(name = "ndcg_metric"),
				tf.keras.metrics.RootMeanSquaredError()
			]
		)

	def call(self, features):
		# We first convert the id features into embeddings.
		# User embeddings are a [batch_size, embedding_dim] tensor.
		user_embeddings = self.user_embeddings(features["user_id"])

		# Movie embeddings are a [batch_size, num_movies_in_list, embedding_dim]
		# tensor.
		movie_embeddings = self.movie_embeddings(features["movie_title"])

		# We want to concatenate user embeddings with movie emebeddings to pass
		# them into the ranking model. To do so, we need to reshape the user
		# embeddings to match the shape of movie embeddings.
		list_length = features["movie_title"].shape[1]
		user_embedding_repeated = tf.repeat(
			tf.expand_dims(user_embeddings, 1), [list_length], axis = 1
		)

		# Once reshaped, we concatenate and pass into the dense layers to generate
		# predictions.
		concatenated_embeddings = tf.concat(
			[user_embedding_repeated, movie_embeddings], 2
		)

		return self.score_model(concatenated_embeddings)

	def compute_loss(self, features, training = False):
		labels = features.pop("user_rating")

		scores = self(features)

		return self.task(
			labels = labels,
			predictions = tf.squeeze(scores, axis = -1),
		)

### Training the models

In [44]:
epochs = 30

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [45]:
mse_model = RankingModel(tf.keras.losses.MeanSquaredError())
mse_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [46]:
mse_model.fit(cached_train, epochs=epochs, verbose=False)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [47]:
hinge_model = RankingModel(tfr.keras.losses.PairwiseHingeLoss())
hinge_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [48]:
hinge_model.fit(cached_train, epochs=epochs, verbose=False)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [49]:
listwise_model = RankingModel(tfr.keras.losses.ListMLELoss())
listwise_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [50]:
listwise_model.fit(cached_train, epochs=epochs, verbose=False)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


### Comparing the models

In [51]:
mse_model_result = mse_model.evaluate(cached_test, return_dict=True)
print("NDCG of the MSE Model: {:.4f}".format(mse_model_result["ndcg_metric"]))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 277ms/step - ndcg_metric: 0.9054 - root_mean_squared_error: 0.9728 - loss: 0.9463 - regularization_loss: 0.0000e+00 - total_loss: 0.9463
NDCG of the MSE Model: 0.9054


In [52]:
hinge_model_result = hinge_model.evaluate(cached_test, return_dict=True)
print("NDCG of the pairwise hinge loss model: {:.4f}".format(hinge_model_result["ndcg_metric"]))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 313ms/step - ndcg_metric: 0.9042 - root_mean_squared_error: 3.7390 - loss: 1.0224 - regularization_loss: 0.0000e+00 - total_loss: 1.0224
NDCG of the pairwise hinge loss model: 0.9042


In [53]:
listwise_model_result = listwise_model.evaluate(cached_test, return_dict=True)
print("NDCG of the ListMLE model: {:.4f}".format(listwise_model_result["ndcg_metric"]))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 311ms/step - ndcg_metric: 0.9063 - root_mean_squared_error: 2.7235 - loss: 4.5405 - regularization_loss: 0.0000e+00 - total_loss: 4.5405
NDCG of the ListMLE model: 0.9063
